In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = DirectoryLoader("/home/jinzy/work/automation/ask_gpt/testdocs", glob="**/*.pdf", show_progress=True, use_multithreading=True)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)
txt_docs = loader.load_and_split(text_splitter=text_splitter)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.58s/it]


In [3]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import GPT4All
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
# llm = HuggingFaceHub(repo_id='THUDM/chatglm-6b', model_kwargs={"trust_remote_code": True})
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10})
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                           model_kwargs={"device": "cuda"})
txt_docsearch = Chroma.from_documents(txt_docs, embeddings)

# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=2048)
# llm = GPT4All(model="/home/jinzy/Downloads/ggml-gpt4all-j-v1.3-groovy.bin", n_ctx=512, n_threads=8)

load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory


system_template = """
Use the following context to answer the user's question.
If you don't know the answer, say you don't, don't try to make it up. 请用中文回答.
-----------
{context}
-----------
{chat_history}
"""

# 构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('{question}')
]

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_messages(messages)

# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt)
doc_chain = load_qa_chain(llm, chain_type="refine")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# chain = ConversationalRetrievalChain(
#     retriever=txt_docsearch.as_retriever(),
#     question_generator=question_generator,
#     combine_docs_chain=doc_chain,
#     memory=memory
# )
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=txt_docsearch.as_retriever(), 
                                              chain_type='refine', memory=memory)

In [5]:
query = "请用中文解释一下RPM产品中的任务挖掘功能"
result = chain({"question": query})
print(result)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_278750/825513186.py:2 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_278750/825513186.py'                        │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/base.py:140 in __call__ │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/base.py:134 in __call__ │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/conversational_retrieva │
│ l/base.py:110 in _call                                                                           │
│                                                                                                  │
│   107 │   │   new_inputs = inputs.copy()                                                         │
│   108 │   │   new_inputs["question"] = new_question                                              │
│   109 │   │   new_inputs["chat_history"] = chat_history_str                                      │
│ ❱ 110 │   │   answer = self.combine_docs_chain.run(                                              │
│   111 │   │   │   input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs         │
│   112 │   │   )                                                                                  │
│   113 │   │   if self.return_source_documents:                                                   │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/base.py:239 in run      │
│                                                                                                  │
│   236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   237 │   │                                                

In [ ]:
query = "请给一个RPM产品的实际案例介绍"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
print(result)
#print(result['answer'])

In [ ]:
print(result['answer'])

In [ ]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": '请用中文回答: ' + query})

print(result['answer'])

In [ ]:
result

In [ ]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query + ' 请用中文回答。'})

print(result['answer'])